# Simulating Continuous Training for the Production MLOps Pipeline

## Overview

__[BLAH BLAH BLAH: This step should only be run once the MLOps Pipeline has been deployed into production as it simulates what happens afterwards.]__

## Section 1 - Setup

><div class="alert alert-block alert-info"><b>NOTE: </b>Recommend using an <em>ml.m5.large</em> (or larger) instance type and, <em>Python 3 (Data Science)</em> kernel to train the <b>CTGAN</b> model.</div

In [ ]:
# Install CTGAN
# !pip install CTGAN pyarrow==2 awswrangler==2.7.0

In [ ]:
from datetime import datetime, timedelta, timezone
import json
import os
import re
import boto3
import io
import requests
import tempfile
import warnings
import pandas as pd
import numpy as np
import time


from time import sleep, gmtime, strftime
from threading import Thread

from sagemaker import get_execution_role, session, Session, image_uris
from sagemaker.s3 import S3Downloader, S3Uploader
from sagemaker.processing import ProcessingJob
from sagemaker.serializers import CSVSerializer
from sagemaker.feature_store.feature_group import FeatureGroup
from sagemaker.model import Model
from sagemaker.model_monitor import DataCaptureConfig


region = boto3.Session().region_name
boto_session = boto3.Session(region_name=region)
sagemaker_client = boto_session.client(service_name='sagemaker', region_name=region)
featurestore_runtime = boto_session.client(service_name='sagemaker-featurestore-runtime', region_name=region)
session = Session()
feature_store_session = Session(
boto_session=boto_session,
sagemaker_client=sagemaker_client,
sagemaker_featurestore_runtime_client=featurestore_runtime
)
s3 = boto3.client('s3')
warnings.filterwarnings('ignore')

In [ ]:
# Get Execution role
role = get_execution_role()
print("RoleArn:", role)

region = session.boto_region_name
print("Region:", region)

In [ ]:
# Pipeline Data bucket
data_bucket = 'data-us-east-2-500842391574'
raw_key = 'input/raw/abalone.csv'
print(f'Raw Data bucket: {data_bucket}')

# Setup S3 bucket parmaters for the production logs bucket
# Enter the name of the Production Logs Bucket, created by the MLOps Pipeline
prod_bucket = 'proddeploymentstage-prodappl-logss3bucket004b0f70-qc1035xby1su'
print(f'Production Logs Bucket: {prod_bucket}')

# S3 prefixes
data_capture_prefix = 'endpoint-data-capture'
s3_capture_upload_path = f's3://{prod_bucket}/{data_capture_prefix}'
ground_truth_upload_path = f's3://{prod_bucket}/ground-truth-data/{datetime.now():%Y-%m-%d-%H-%M-%S}'

# Get the model monitor image
monitor_image_uri = image_uris.retrieve(framework="model-monitor", region=region)

print(f'Image URI: {monitor_image_uri}')
print(f'Capture path: {s3_capture_upload_path}')
print(f'Ground truth path: {ground_truth_upload_path}')

---

## Section 2 - Review Baseline Data

We will re-create the Model Quality baseline job (even though it was already created by the CDK Pipeline) to see the output of the SageMaker SDK when calling `create_monitoring_schedule()`, as well as, to leverage the resultant constraints when creating the monitoring schedule. __[WRONG]__ The baseline suggestion is now part of the Production Model deployment.

In [ ]:
# Set up the locations for capturing the baseline results
# This should already be in place from the CDK Pipeline,
# with the `baseline.csv` already there
baseline_prefix = 'baselining'
baseline_data_prefix = baseline_prefix + '/data'
baseline_results_prefix = baseline_prefix + '/results'

baseline_dataset_uri = f's3://{prod_bucket}/{baseline_data_prefix}'
baseline_results_uri = f's3://{prod_bucket}/{baseline_results_prefix}'
print(f'Baseline data uri: {baseline_dataset_uri}')
print(f'Baseline results uri: {baseline_results_uri}')

### Explore the Generated Metrics

__[Thes shoudl match what's in the Model Registry]__

In [ ]:
statistics_obj = s3.get_object(Bucket=prod_bucket, Key=f'{baseline_results_prefix}/statistics.json')
statistics_json = json.loads(statistics_obj['Body'].read().decode('utf-8'))['regression_metrics']
pd.json_normalize(statistics_json).T

### Explore the Generated Constraints

__[Explain what these Constraints represent]__

In [ ]:
constraints_obj = s3.get_object(Bucket=prod_bucket, Key=f'{baseline_results_prefix}/constraints.json')
constraints_json = json.loads(constraints_obj['Body'].read().decode('utf-8'))['regression_constraints']
pd.json_normalize(constraints_json).T

---

## Section 3 - Create Inferene Data to Test the Model Quality Monitor

Model Quality Monitoring needs two additional inputs - predictions made by the deployed model endpoint and the ground truth data to be provided by the model consuming application. Since you already enabled data capture on the endpoint, prediction data is captured in S3. The ground truth data depends on the what the model is predicting and what the business use case is.

### Gerating Synthetic Abalone data

In order to generate prediction data we will need to create fake "new" data. To accomplish this, we will use the CTGAN package and train it on the "raw" abaloen dataset. We will create $1000$ samples of fake data.

__[Why $1000$ samples?]__


><div class="alert alert-block alert-info"><b>NOTE: </b>When adding <em>1000</em> samples to the 'raw' data without reshuffling, the model performance drastically underfits the <em>testing.csv</em> dataset. To create more data variance, the 'raw' data and 'feature store' data is shuffled.</div>

__[Why CTGAN?]__

In [ ]:
# # 'raw' data column names
# names = [
#     'sex',
#     'length',
#     'diameter',
#     'height',
#     'whole_weight',
#     'shucked_weight',
#     'viscera_weight',
#     'shell_weight',
#     'rings'
# ]

# # Location of the 'raw' data
# obj = s3.get_object(Bucket=data_bucket, Key=raw_key)
# raw_data = pd.read_csv(io.BytesIO(obj['Body'].read()), encoding='utf8', names=names)
# raw_data.head()

>__NOTE:__ CTGAN training should take around 15 minutes.

In [ ]:
# from ctgan import CTGANSynthesizer

# # Fit the CTGAN model, declaring the 'sex' and 'rings' columns as discrete variables
# ctgan = CTGANSynthesizer()
# ctgan.fit(raw_data, ['sex', 'rings'], epochs=1000)

In [ ]:
# # Generate 1000 samples from the CTGAN model
# samples = ctgan.sample(200)

In [ ]:
# # Compare the raw data
# raw_data.describe()

In [ ]:
# # Compare the sample data
# samples.describe()

In [ ]:
# # Save the samples as fake abalone data
# samples.to_csv('fake-abalone.csv', header=False, index=False)

### Pre-processing the Synthetic Abalone data for Continuous Model Training

When the Model Quality Monitor (See Section 5) determines that model re-training is necessary, we will need "new" data to facilitate this. Along with using the Synthetic Abalone data to simulate user inferences, we will also make this data available to the MLOPs Pipeline for Continuos Training. Amazon SageMaker provides a fully managed, purpose-built repository to store the processed features in the form of the [Amazon SageMaker Feature Store](https://aws.amazon.com/sagemaker/feature-store/).

><div class="alert alert-block alert-info"><b>NOTE: </b>The following is code cell duplicates the functionality performed by the <em>preprocessing.py</em> script in the MLOps Pipeline.</div>

__[Why are doing this?]__

In [ ]:
# from sklearn.compose import ColumnTransformer
# from sklearn.impute import SimpleImputer
# from sklearn.pipeline import Pipeline
# from sklearn.preprocessing import StandardScaler, OneHotEncoder


# # Since we get a headerless CSV file we specify the column names here.
# feature_columns_names = [
#     'sex',
#     'length',
#     'diameter',
#     'height',
#     'whole_weight',
#     'shucked_weight',
#     'viscera_weight',
#     'shell_weight',
# ]
# label_column = 'rings'

# feature_columns_dtype = {
#     'sex': str,
#     'length': np.float64,
#     'diameter': np.float64,
#     'height': np.float64,
#     'whole_weight': np.float64,
#     'shucked_weight': np.float64,
#     'viscera_weight': np.float64,
#     'shell_weight': np.float64
# }
# label_column_dtype = {'rings': np.float64}


# def merge_two_dicts(x, y):
#     z = x.copy()
#     z.update(y)
#     return z


# df = pd.read_csv(
#     'fake-abalone.csv',
#     header=None, 
#     names=feature_columns_names + [label_column],
#     dtype=merge_two_dicts(feature_columns_dtype, label_column_dtype)
# )

# numeric_features = list(feature_columns_names)
# numeric_features.remove('sex')
# numeric_transformer = Pipeline(
#     steps=[
#         ('imputer', SimpleImputer(strategy='median')),
#         ('scaler', StandardScaler())
#     ]
# )

# categorical_features = ['sex']
# categorical_transformer = Pipeline(
#     steps=[
#         ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
#         ('onehot', OneHotEncoder(handle_unknown='ignore'))
#     ]
# )

# preprocess = ColumnTransformer(
#     transformers=[
#         ('num', numeric_transformer, numeric_features),
#         ('cat', categorical_transformer, categorical_features)
#     ]
# )
    
# y = df.pop('rings')
# X_pre = preprocess.fit_transform(df)
# y_pre = y.to_numpy().reshape(len(y), 1)
# X = np.concatenate((y_pre, X_pre), axis=1)
# np.random.shuffle(X)

# new_header = [
#     'rings',
#     'length',
#     'diameter',
#     'height',
#     'whole_weight',
#     'shucked_weight',
#     'viscera_weight',
#     'shell_weight',
#     'sex_F',
#     'sex_I',
#     'sex_M'
# ]
# new_training_data = pd.DataFrame(X, columns=new_header)

# # View the the new, pre-processed training data
# new_training_data.head()

### Create the FeatureStore

In [ ]:
# current_time_sec = int(round(time.time()))

# feature_group_name = 'AbaloneFeatureGroup'

# abalone_feature_group = FeatureGroup(
#     name=feature_group_name,
#     sagemaker_session=feature_store_session
# )

# record_identifier_feature_name = 'rings'
# event_time_feature_name = 'EventTime'

In [ ]:
# new_training_data[event_time_feature_name] = pd.Series([current_time_sec]*len(new_training_data), dtype='float64')
# new_training_data[event_time_feature_name] = pd.Series([current_time_sec]*len(new_training_data), dtype='float64')
# new_training_data

In [ ]:
# abalone_feature_group.load_feature_definitions(
#     data_frame=new_training_data
# )

In [ ]:
# def wait_for_feature_group_creation_complete(feature_group):
#     status = feature_group.describe().get('FeatureGroupStatus')
#     while status == 'Creating':
#         print('Waiting for Feature Group Creation')
#         time.sleep(5)
#         status = feature_group.describe().get('FeatureGroupStatus')
#     if status != 'Created':
#         raise RuntimeError(f'Failed to create feature group {feature_group.name}')
#     print(f'FeatureGroup {feature_group.name} successfully created.')

# abalone_feature_group.create(
#     s3_uri=f's3://{data_bucket}/featurestore',
#     record_identifier_name=record_identifier_feature_name,
#     event_time_feature_name=event_time_feature_name,
#     role_arn=role,
#     enable_online_store=True
# )

# wait_for_feature_group_creation_complete(feature_group=abalone_feature_group)

#### Confirm FeatureStore Creation

In [ ]:
# abalone_feature_group.describe()

### Ingest Pre-Processed Synthetic Training Data

In [ ]:
# abalone_feature_group.ingest(data_frame=new_training_data, max_workers=5, wait=True)

#### Confirm Data Ingest

><div class="alert alert-block alert-info"><b>NOTE: </b>Data ingestion should take around <em>6 - 7</em> Minutes.</div>

In [ ]:
# feature_group_resolved_output_s3_uri = abalone_feature_group.describe().get("OfflineStoreConfig").get("S3StorageConfig").get("ResolvedOutputS3Uri")
# feature_group_s3_prefix = feature_group_resolved_output_s3_uri.replace(f"s3://{data_bucket}/", "")
# offline_store_contents = None
# while (offline_store_contents is None):
#     objects_in_bucket = s3.list_objects(Bucket=data_bucket,Prefix=feature_group_s3_prefix)
#     if ('Contents' in objects_in_bucket and len(objects_in_bucket['Contents']) > 1):
#         offline_store_contents = objects_in_bucket['Contents']
#     else:
#         print('Waiting for data in offline store...\n')
#         sleep(60)

# print('Data available.')

In [ ]:
# # Create the Athena query instance
# query = abalone_feature_group.athena_query()

# # Get the name of the table to query
# table = query.table_name

# # Select the columns to get results
# cols = [
#     'rings',
#     'length',
#     'diameter',
#     'height',
#     'whole_weight',
#     'shucked_weight',
#     'viscera_weight',
#     'shell_weight',
#     'sex_F',
#     'sex_I',
#     'sex_M'
# ]

# # Create the SQL Query
# query_string = f'SELECT {",".join(cols)} FROM "{table}"'

# # Execute the query against Athena
# query.run(query_string=query_string, output_location=f's3://{data_bucket}/query_results/')
# query.wait()

# # View Query results as a pandas DataFrame
# results = query.as_dataframe()
# results

---

## Section 4 - Setup Continuous Model Monitoring to identify model quality drift 

### Processing the Synthetic Abalone Data for User Inference Simulation

Unlike the origional 'raw' dataset, we will not be performing any numerical pre-processing, as we need to create simualted raw data as user inference data. We only want to re-structure the inference data into a format that resembles the format used by the Website Form. This way, the inference requests will effectivley simulate users submit prediction requests using the Web Form. We will be using $300$ rndom samples.

__[Why $300$ samples?]__

In [ ]:
# import random

# # Since we get a headerless CSV file we specify the column names here.
# feature_columns_names = [
#     'sex',
#     'length',
#     'diameter',
#     'height',
#     'whole_weight',
#     'shucked_weight',
#     'viscera_weight',
#     'shell_weight',
# ]
# label_column = 'rings'

# feature_columns_dtype = {
#     'sex': str,
#     'length': np.float64,
#     'diameter': np.float64,
#     'height': np.float64,
#     'whole_weight': np.float64,
#     'shucked_weight': np.float64,
#     'viscera_weight': np.float64,
#     'shell_weight': np.float64
# }
# label_column_dtype = {'rings': np.float64}


# def merge_two_dicts(x, y):
#     z = x.copy()
#     z.update(y)
#     return z


# # Rstructure the data for inference for only 300 random samples
# # n = 1000 # size of the dataset
# # s = 200 # numer of tamples to take
# # skip = sorted(random.sample(range(n), n-s))
# df = pd.read_csv(
#     'fake-abalone.csv',
#     names=feature_columns_names + [label_column],
#     dtype=merge_two_dicts(feature_columns_dtype, label_column_dtype),
# #     skiprows=skip
# )

# # Separate the labels
# y = df.pop('rings')

# # Reorder colums to match inference format
# cols = df.columns.values 
# reordered_cols = [
#     'length',
#     'diameter',
#     'height',
#     'whole_weight',
#     'shucked_weight',
#     'viscera_weight',
#     'shell_weight',
#     'sex'
# ]
# x = df.reindex(columns=reordered_cols)

# # Create the inference dataset
# x.to_csv('inference-data.csv', index=False)

# # Create the ground truth dataset
# y.to_csv('ground-truth.csv', header=['label'], index=False)

We should now have the following syntehtic data files:
- `inference-data.csv`: Preprocessed data to generate predictions from the Production Endpoint.
- `ground-truth.csv`: The ground truth labels from the synthetic data wich to compare the quality of the model's predictions.

### Generate Inferences using the `FormProcessingAPI`

In [ ]:
# Endpoint name
endpoint_name = 'abalone-prod-endpoint'

# Form API Enpoint
api_url = 'https://10r26vbp95.execute-api.us-east-2.amazonaws.com/'+'api/predict'

def invoke_api(url, file_name):
    df = pd.read_csv(file_name)
    i = 0
    for row in range(len(df)):
        headers = {"content-type":"application/json; charset=UTF-8", "inference-id": str(i)}
        body = json.loads(df.iloc[row].to_json())
        response = requests.post(url, headers=headers, data=json.dumps(body))
        i += 1
        sleep(1)
            
def invoke_api_forever():
    while True:
        invoke_api(api_url, 'inference-data.csv')
        
api_thread = Thread(target=invoke_api_forever)
api_thread.start()

View captured data stored in Amazon S3. You should expect to see different files from different time periods organized based on the hour in which the invocation occurred. The format of the Amazon S3 path is:

`s3://{destination-bucket-prefix}/{endpoint-name}/{variant-name}/yyyy/mm/dd/hh/filename.jsonl`

In [ ]:
print('Waiting for captures to show up', end='')
for _ in range(120): #2 Minutes
    capture_files = sorted(S3Downloader.list(f'{s3_capture_upload_path}/{endpoint_name}'))
    if capture_files:
        capture_file = S3Downloader.read_file(capture_files[-1]).split('\n')
        capture_record = json.loads(capture_file[0])
        if 'inferenceId' in capture_record['eventMetadata']:
            break
    print('.', end='', flush=True)
    sleep(1)
print()
print('Found Capture Files:')
print('\n '.join(capture_files[-3:]))

View the contents of a single capture file. Here you should see all the data captured in an Amazon SageMaker specific JSON-line formatted file. Take a quick peek at the first few lines in the captured file.

In [ ]:
print('\n'.join(capture_file[-3:-1]))

View the contents of a single line is present below in a formatted JSON file so that you can observe a little better.

><div class="alert alert-block alert-info"><b>NOTE: </b>Take note of the <em>inferenceId</em>.</div>

In [ ]:
print(json.dumps(capture_record, indent=2))

### Generating Synthetic Ground Truth

In [ ]:
df = pd.read_csv('ground-truth.csv')
NUM_GROUND_TRUTH_RECORDS = len(df)

def ground_truth_with_id(inference_id):
    label = round(df.iloc[inference_id][0])
    return {
        'groundTruthData': {
            'data': str(label),
            'encoding': 'CSV'
        },
        'eventMetadata': {
            'eventId': str(inference_id),
        },
        'eventVersion': '0',
    }

def upload_ground_truth(records, upload_time):
    fake_records = [ json.dumps(r) for r in records ]
    data_to_upload = '\n'.join(fake_records)
    target_s3_uri = f'{ground_truth_upload_path}/{upload_time:%Y/%m/%d/%H/%M%S}.jsonl'
    print(f'Uploading {len(fake_records)} records to', target_s3_uri)
    S3Uploader.upload_string_as_file_body(data_to_upload, target_s3_uri)

def generate_fake_ground_truth_forever():
    j = 0
    while True:
        fake_records = [ ground_truth_with_id(i) for i in range(NUM_GROUND_TRUTH_RECORDS) ]
        upload_ground_truth(fake_records, datetime.utcnow())
        j = (j + 1) % 5
        sleep(60*60) # do this once an hour

gt_thread = Thread(target=generate_fake_ground_truth_forever)
gt_thread.start()

---

# Appendix A: Manual process for Model Quality Monitoring

## Quality Baselining

### Manually Create Model Quality Baseline using the SageMaker SDK

In [ ]:
# Generate a new baseline job
from sagemaker.model_monitor import ModelQualityMonitor
from sagemaker.model_monitor import EndpointInput
from sagemaker.model_monitor.dataset_format import DatasetFormat

# Create the model quality monitoring object
model_quality_monitor = ModelQualityMonitor(
    role=role,
    instance_count=1,
    instance_type='ml.m5.xlarge',
    volume_size_in_gb=20,
    max_runtime_in_seconds=1800,
    sagemaker_session=session
)

# Name of the model quality baseline job
baseline_job_name = f'abalone-baseline-{datetime.utcnow():%Y-%m-%d-%H%M}'

### Manually Suggest a Model Quality Baseline using the SageMaker SDK

In [ ]:
# Execute the baseline suggestion job. 
# Specify problem type, in this case Regression, and provide other required attributes.
job = model_quality_monitor.suggest_baseline(
    job_name=baseline_job_name,
    baseline_dataset=baseline_dataset_uri,
    dataset_format=DatasetFormat.csv(header=True),
    output_s3_uri = baseline_results_uri,
    problem_type='Regression',
    inference_attribute= "prediction",
    ground_truth_attribute= "label"
)
job.wait(logs=False)

In [ ]:
baseline_job = model_quality_monitor.latest_baselining_job

In [ ]:
regression_metrics = baseline_job.baseline_statistics().body_dict["regression_metrics"]
pd.json_normalize(regression_metrics).T

In [ ]:
pd.DataFrame(baseline_job.suggested_constraints().body_dict["regression_constraints"]).T

In [ ]:
baseline_job.suggested_constraints().file_s3_uri

## Model Quaility Monitoring Schedule

### Manually Create a Model Quality Monitoring Schedule

In [ ]:
# Endpoint name
endpoint_name = 'abalone-prod-endpoint'

# Monitoring schedule name
monitor_schedule_name = f'abalone-monitoring-schedule-{datetime.utcnow():%Y-%m-%d-%H%M}'

In [ ]:
#Create an enpointInput 
endpointInput = EndpointInput(endpoint_name=endpoint_name, 
                              inference_attribute='0',
                              destination='/opt/ml/processing/input_data')

In [ ]:
from sagemaker.model_monitor import CronExpressionGenerator

response = model_quality_monitor.create_monitoring_schedule(
    monitor_schedule_name=monitor_schedule_name,
    endpoint_input=endpointInput,
    output_s3_uri = baseline_results_uri,
    problem_type='Regression',
    ground_truth_input=ground_truth_upload_path,
    constraints=f'{baseline_results_uri}/constraints.json',
    schedule_cron_expression=CronExpressionGenerator.hourly(), 
    enable_cloudwatch_metrics=True
)

In [ ]:
#Create the monitoring schedule
#You will see the monitoring schedule in the 'Scheduled' status
model_quality_monitor.describe_schedule()

In [ ]:
#Initially there will be no executions since the first execution happens at the top of the hour
#Note that it is common for the execution to luanch upto 20 min after the hour.
executions = model_quality_monitor.list_executions()
executions

#### Wait for the first execution of the Monitoring Schedule

>__NOTE:__ This can take between $15$ and $20$ minutes past the top of the hour.

In [ ]:
# Wait for the first execution of the monitoring_schedule
print('Waiting for first execution', end='')
while True:
    execution = model_quality_monitor.describe_schedule().get('LastMonitoringExecutionSummary')
    if execution:
        break
    print('.', end='', flush=True)
    sleep(10)
print()
print('Execution found!')


In [ ]:
while not executions:
    executions = model_quality_monitor.list_executions()
    sleep(10)
latest_execution = executions[-1]
latest_execution.describe()

#### Review the Monitoring Schedule Output

In [ ]:
status = execution['MonitoringExecutionStatus']

while status in ['Pending', 'InProgress']:
    print('Waiting for execution to finish', end='')
    latest_execution.wait(logs=False)
    latest_job = latest_execution.describe()
    print()
    print(f"{latest_job['ProcessingJobName']} job status:", latest_job['ProcessingJobStatus'])
    print(f"{latest_job['ProcessingJobName']} job exit message, if any:", latest_job.get('ExitMessage'))
    print(f"{latest_job['ProcessingJobName']} job failure reason, if any:", latest_job.get('FailureReason'))
    sleep(30) # model quality executions consist of two Processing jobs, wait for second job to start
    latest_execution = model_quality_monitor.list_executions()[-1]
    execution = model_quality_monitor.describe_schedule()['LastMonitoringExecutionSummary']
    status = execution['MonitoringExecutionStatus']

print(f'Execution status is: {status}')
    
if status != 'Completed':
    print(execution)
    print('====STOP==== \n No completed executions to inspect further. Please wait till an execution completes or investigate previously reported failures.')


In [ ]:
latest_execution = model_quality_monitor.list_executions()[-1]
report_uri = latest_execution.describe()['ProcessingOutputConfig']['Outputs'][0]['S3Output']['S3Uri']
print(f'Report Uri: {report_uri}')

In [ ]:
pd.options.display.max_colwidth = None
violations = latest_execution.constraint_violations().body_dict['violations']
violations_df = pd.json_normalize(violations)
violations_df.head(10)

## Model Quality CloudWatch Metrics

### List the Generated CloudWatch Metrics

In [ ]:
# # Create CloudWatch client
# cw_client = boto3.Session().client('cloudwatch')

# namespace = f'aws/sagemaker/Endpoints/model-metrics'

# cw_dimensions=[
#         {
#             'Name': 'Endpoint',
#             'Value': endpoint_name
#         },
#         {
#             'Name': 'MonitoringSchedule',
#             'Value': monitor_schedule_name
#         }
# ]

In [ ]:
# # List metrics through the pagination interface
# paginator = cw_client.get_paginator('list_metrics')

# for response in paginator.paginate(Dimensions=cw_dimensions,Namespace=namespace):
#     model_quality_metrics = response['Metrics']
#     for metric in model_quality_metrics:
#         print(metric['MetricName'])

### Create the CloudWatch Alarm

In [ ]:
# Create CloudWatch client
cw_client = boto3.Session().client('cloudwatch')

alarm_name='MODEL_QUALITY_ALARM'
alarm_desc='Trigger an CloudWatch alarm when the rmse score drifts away from the baseline constraints'

# Setting the threshold to match the rmse threshold from the baseline evaluation
model_quality_rmse_threshold=3.1
metric_name='rmse'
namespace = f'aws/sagemaker/Endpoints/model-metrics'

cw_client.put_metric_alarm(
    AlarmName=alarm_name,
    AlarmDescription=alarm_desc,
    ActionsEnabled=True,
    MetricName=metric_name,
    Namespace=namespace,
    Statistic='Average',
    Dimensions=[
        {
            'Name': 'Endpoint',
            'Value': endpoint_name
        },
        {
            'Name': 'MonitoringSchedule',
            'Value': monitor_schedule_name
        }
    ],
    Period=5400,
    EvaluationPeriods=1,
    DatapointsToAlarm=1,
    Threshold=model_quality_rmse_threshold,
    ComparisonOperator='GreaterThanThreshold',
    TreatMissingData='missing'
#     TreatMissingData='breaching'
)

## Cleanup

In [ ]:
model_quality_monitor.delete_monitoring_schedule()